In [1]:
from polygon import RESTClient
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

client = RESTClient("hh0mSNz8kJH3asJCwAscTrtLt4RYb5RM")

def calculate_ema(data, period):
  return data.ewm(span=period, adjust=False).mean()

def calculate_macd(data):
  ema_12 = calculate_ema(data, 12)
  ema_26 = calculate_ema(data, 26)
  macd_line = ema_12 - ema_26
  signal_line = calculate_ema(macd_line, 9)
  return macd_line, signal_line

def generate_date_array(year):
    dates = []
    # Start from the first day of the given year
    current_date = datetime(year, 1, 1)
    # End on the last day of the given year
    end_date = datetime(year + 1, 1, 1)
    
    while current_date < end_date:
        # Format the date as "yyyy-mm-dd" and add it to the list
        date_string = current_date.strftime("%Y-%m-%d")
        dates.append(date_string)
        # Move to the next day
        current_date += timedelta(days=1)
    
    return dates

def graph_profits(data, type_of_profit):
    # Extract keys and values
    dates = list(data.keys())
    values = list(data.values())

    # Convert string dates to datetime objects
    dates = [datetime.strptime(date, '%Y-%m-%d') for date in dates]

    # Create a plot
    plt.figure(figsize=(8, 4))
    plt.plot(dates, values, marker='o', linestyle='-', color='b')

    # Format the x-axis to show dates properly
    plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))
    plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.DayLocator())

    # Label the axes
    plt.xlabel('Date')
    plt.ylabel(type_of_profit)
    plt.title(f'Model {type_of_profit} vs Holding Stock')

    # Rotate date labels for better readability
    plt.xticks(rotation=45)

    # Show grid
    plt.grid(True)

    # Display the plot
    plt.tight_layout()
    plt.show()

def run_simulation(ticker, day, init_cash, init_shares, percent_invest, prices, times, buys, sells):
  transaction_times = sorted(buys + sells)
  transactions = []
  for t in transaction_times:
    price = prices[times.index(t)]
    if t in buys:
      transactions.append([t, price, "Buy"])
    else:
      transactions.append([t, price, "Sell"])
  # Execute Transactions through simulation
  cash = init_cash
  shares = init_shares
  # print("Running Simulation for MACD with", ticker, "on", day)
  # print("Number of Transactions: ", len(transactions))
  # print("Buys: ", len(buys), "Sells: ", len(sells))
  # print("Open Price, Close Price: ", prices[0], prices[-1])
  # print("BOD Cash and Shares: ", cash, shares)
  portfolio_val_s = cash + prices[0] * shares
  # print("BOD Portfolio Value: ", portfolio_val_s)
  amt = cash * percent_invest / 100
  for order in transactions:
    if order[2] == "Buy":
      cash -= amt
      shares += amt/order[1]
    else:
      cash += amt
      shares -= amt/order[1]
  # print("EOD Cash and Shares: ", cash, shares)
  portfolio_val_e = cash + prices[-1] * shares
  profit = portfolio_val_e - portfolio_val_s
  profit_percentage = (portfolio_val_e - portfolio_val_s) / portfolio_val_s * 100

  base_profit_percentage = init_shares*(prices[-1]-prices[0])/portfolio_val_s * 100

  # print("EOD Portfolio Value: ", portfolio_val_e)
  # print("Model Profit Percentage: ", profit_percentage)

  # print("Market Change w/o trades", base_profit_percentage)
  # print("Model outperforms market by", (profit_percentage - base_profit_percentage), "%") 


  return [cash, shares, profit, profit_percentage, base_profit_percentage]



In [2]:
def invest_macd(ticker, day, cash=100000, shares=1000, percent_invest=20):
  aggs = client.get_aggs(ticker, 1, "minute", day, day)
  timestamps = []
  close_prices = []
  for agg in aggs:
    timestamps.append(datetime.fromtimestamp(agg.timestamp // 1000))
    close_prices.append(agg.close)
  close_prices = np.array(close_prices)
  x_close = []
  y_close = []
  for i in range(0, len(aggs), 1):
    agg = aggs[i]
    x_close.append(datetime.fromtimestamp(agg.timestamp // 1000).time().strftime("%H:%M"))
    y_close.append(agg.close)

  #check if there is data for this day
  # print(len(y_close))
  if len(y_close) == 0:
    print(f"no data for this day {day} and stock")
    return(cash, shares, 0, 0, 0)
  
  # Convert close prices to Pandas Series
  close_series = pd.Series(close_prices)
  # Calculate MACD and Signal line
  macd_line, signal_line = calculate_macd(close_series)
  # Calculate short-term EMAs
  ema_5 = calculate_ema(close_series, 5)
  ema_10 = calculate_ema(close_series, 10)
  ema_12 = calculate_ema(close_series, 12)
  ema_20 = calculate_ema(close_series, 20)
  ema_26 = calculate_ema(close_series, 26)
    # Plot closing prices (debug):
    
  # plt.figure(figsize=(14, 7))
  # plt.plot(timestamps, close_prices, label='Close Prices')
  # plt.plot(timestamps, ema_5, label='5-period EMA', linestyle='--')
  # plt.plot(timestamps, ema_10, label='10-period EMA', linestyle='--')
  # plt.plot(timestamps, ema_12, label='12-period EMA', linestyle='--')
  # plt.plot(timestamps, ema_20, label='20-period EMA', linestyle='--')
  # plt.plot(timestamps, ema_26, label='26-period EMA', linestyle='--')
  # plt.title('Closing Prices and Short-term EMAs')
  # plt.legend()
  # plt.show()
  # plt.figure(figsize=(14, 7))
  # plt.plot(timestamps, macd_line, label='MACD Line', color='blue')
  # plt.plot(timestamps, signal_line, label='Signal Line', color='red')

  buys = []
  sells = []
  # Highlight crossovers (debug)
  for i in range(1, len(macd_line)):
    if (macd_line[i-1] < signal_line[i-1]) and (macd_line[i] > signal_line[i]):
      # plt.axvline(x=timestamps[i], color='green', linestyle='--', linewidth=0.5)  # Bullish crossover
      buys.append(x_close[i])
    elif (macd_line[i-1] > signal_line[i-1]) and (macd_line[i] < signal_line[i]):
      # plt.axvline(x=timestamps[i], color='red', linestyle='--', linewidth=0.5)  # Bearish crossover
      sells.append(x_close[i])
      
  # plt.title('MACD and Signal Line with Crossovers')
  # plt.legend()

  # plt.show()

  return run_simulation(ticker, day, cash, shares, percent_invest, y_close, x_close, buys, sells)   


In [3]:
def test_model(ticker, start_date, end_date, init_cash, init_shares, percent_invest):
    year1 = int(start_date[:4])
    year2 = int(end_date[:4])
    date_array = []
    for year in range(year1, year2+1):
        date_array += generate_date_array(year)

    alphas = []
    alphas_dict = {}
    profits_dict = {}
    start_d_idx = date_array.index(start_date)
    end_d_idx = date_array.index(end_date)
    cash = init_cash
    shares = init_shares

    for date in date_array[start_d_idx: end_d_idx+1]: #has to be 4 because 
        # print(date)
        cash, shares, profit, profit_percentage, base_profit_percentage = invest_macd(ticker, date, cash, shares, percent_invest)
        alphas.append(alpha:=(profit_percentage-base_profit_percentage))
        profits_dict[date] = profit_percentage
        alphas_dict[date] = alpha
        # print()

    # print(alphas_dict)

    return alphas_dict, profits_dict

#test but only 4 days allowed in free API
alphas_dict, profits_dict = test_model("AMZN", "2023-01-11", "2023-03-14", 100000, 1000, 20) 

MaxRetryError: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v2/aggs/ticker/AMZN/range/1/minute/2023-01-11/2023-01-11 (Caused by ResponseError('too many 429 error responses'))

In [ ]:
graph_profits(profits_dict, "Profit")
graph_profits(alphas_dict, "Alpha")
print(f"Average Profit: {sum(profits_dict.values())/len(profits_dict.values())}")
print(f"Average Alpha: {sum(alphas_dict.values())/len(alphas_dict.values())}")
